In [26]:
import numpy as np
import cv2
import random
import os
import shutil
from sklearn.model_selection import train_test_split

In [27]:
USER_NAME = "jbw"  
ROOT_DIR = f"/home/{USER_NAME}/yolo_fashion_mnist"
RAW_DATA_DIR = f"{ROOT_DIR}/raw_data"  # 原始数据位置
YOLO_DATA_DIR = f"{ROOT_DIR}/yolo_dataset"  # YOLO数据集保存位置
IMG_SIZE = 128  # 合成图片尺寸（128x128）
TOTAL_SAMPLES = 2000  # 生成2000个合成样本
# Fashion MNIST类别名称（0-9对应）
CLASS_NAMES = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]


In [28]:
def load_mnist_images(path):
    """加载图片数据(Fashion MNIST格式和MNIST完全一致)"""
    with open(path, "rb") as f:
        # 前16字节：魔数、样本数、行数、列数（大端序4字节整数）
        magic, num, rows, cols = np.frombuffer(f.read(16), dtype=">i4")
        # 剩余字节：像素数据（0-255灰度值）
        images = np.frombuffer(f.read(), dtype=np.uint8)
        # 重塑为[样本数, 28, 28]
        images = images.reshape(num, rows, cols)
    return images

def load_mnist_labels(path):
    """加载标签数据"""
    with open(path, "rb") as f:
        # 前8字节：魔数、标签数
        magic, num = np.frombuffer(f.read(8), dtype=">i4")
        # 剩余字节：标签（0-9）
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

In [29]:
train_images = load_mnist_images(f"{RAW_DATA_DIR}/train-images-idx3-ubyte")
train_labels = load_mnist_labels(f"{RAW_DATA_DIR}/train-labels-idx1-ubyte")
print(f"加载成功！训练集图片形状：{train_images.shape}(60000张28x28图片)")

加载成功！训练集图片形状：(60000, 28, 28)(60000张28x28图片)


In [30]:
def sample_fashion_mnist():
    """随机抽取一张Fashion MNIST图片+对应标签"""
    idx = random.randint(0, len(train_images)-1)
    return train_images[idx], int(train_labels[idx])

In [31]:
def generate_sample(num_items=2):
    """
    生成合成图：
    - num_items：合成的时尚单品数量（默认2个）
    - 返回：合成画布、YOLO格式标签列表
    """
    # 创建128x128黑色画布（0=黑色，uint8=0-255）
    canvas = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)
    labels = []

    for _ in range(num_items):
        # 随机抽取一张单品图片+标签
        img, cls = sample_fashion_mnist()
        h, w = img.shape  # 28x28

        # 随机位置（确保单品不超出画布）
        x = random.randint(0, IMG_SIZE - w)
        y = random.randint(0, IMG_SIZE - h)
        # 将单品粘贴到画布上
        canvas[y:y+h, x:x+w] = img

        # 计算YOLO格式标注（归一化中心坐标+宽高，范围0-1）
        cx = (x + w/2) / IMG_SIZE  # 中心x
        cy = (y + h/2) / IMG_SIZE  # 中心y
        bw = w / IMG_SIZE          # 宽度
        bh = h / IMG_SIZE          # 高度

        # YOLO标签格式：类别索引 中心x 中心y 宽度 高度（保留6位小数）
        labels.append(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
    
    return canvas, labels

In [32]:
# 训练集图片/标签保存路径
TRAIN_IMG_DIR = f"{YOLO_DATA_DIR}/images/train"
TRAIN_LBL_DIR = f"{YOLO_DATA_DIR}/labels/train"
# 创建文件夹（不存在则创建，存在不报错）
os.makedirs(TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(TRAIN_LBL_DIR, exist_ok=True)

# 生成2000个合成样本
for i in range(TOTAL_SAMPLES):
    # 随机生成1-3个单品的合成图
    img, lbls = generate_sample(random.randint(1, 3))
    # 保存图片（JPG格式）
    cv2.imwrite(f"{TRAIN_IMG_DIR}/{i}.jpg", img)
    # 保存标签（TXT格式，每行一个目标）
    with open(f"{TRAIN_LBL_DIR}/{i}.txt", "w") as f:
        f.write("\n".join(lbls))

In [33]:
# 验证集文件夹
VAL_IMG_DIR = f"{YOLO_DATA_DIR}/images/val"
VAL_LBL_DIR = f"{YOLO_DATA_DIR}/labels/val"
os.makedirs(VAL_IMG_DIR, exist_ok=True)
os.makedirs(VAL_LBL_DIR, exist_ok=True)

# 获取所有训练集图片
all_imgs = [f for f in os.listdir(TRAIN_IMG_DIR) if f.endswith(".jpg")]
# 划分训练/验证集
train_imgs, val_imgs = train_test_split(all_imgs, test_size=0.2, random_state=42)

# 移动验证集图片和标签
for img_name in val_imgs:
    # 移动图片
    src_img = f"{TRAIN_IMG_DIR}/{img_name}"
    dst_img = f"{VAL_IMG_DIR}/{img_name}"
    shutil.move(src_img, dst_img)
    # 移动对应标签（替换.jpg为.txt）
    lbl_name = img_name.replace(".jpg", ".txt")
    src_lbl = f"{TRAIN_LBL_DIR}/{lbl_name}"
    dst_lbl = f"{VAL_LBL_DIR}/{lbl_name}"
    if os.path.exists(src_lbl):
        shutil.move(src_lbl, dst_lbl)

print(f"数据集生成完成！位置：{YOLO_DATA_DIR}")
print(f"- 训练集：{len(train_imgs)}张图片 | 验证集：{len(val_imgs)}张图片")

数据集生成完成！位置：/home/jbw/yolo_fashion_mnist/yolo_dataset
- 训练集：1600张图片 | 验证集：400张图片
